In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from torch import nn
from tqdm import tqdm

import mlx.core as mx

os.environ["CONFIG_PATHS"] = "../configs/self_play.yaml"
os.environ["CONFIG_OVERRIDES"] = 'game.moves_directory="../data/moves_10"'
sys.path.append("../src")

from configuration import config
from neural_net import NeuralNet
from neural_net_mlx import NeuralNetMLX
""

Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 0, "display_logs_in_console": false, "output_directory": "data/2024-12-06_00-09-14-self-play-for-policy-weight-one"}, "logging": {"save_interval": 3600, "mcts_report_fraction": 0, "ucb_report": false, "gpu_evaluation": true, "made_move": true}, "game": {"board_size": 10, "num_moves": 6233, "num_pieces": 21, "moves_directory": "../data/moves_10"}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256, "game_flush_threshold": 200}, "training": {"run": true, "network_name": "default", "batch_size": 64, "policy_loss_weight": 1.0, "learning_rate": 0.001, "sample_window": 50000, "samples_per_generation": 10000, "sampling_ratio": 2.0, "minimum_window_size": 10000, "new_data_check_interval": 60}, "networks": {"default": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8, "model_path": "", "model_directory": "d

''

In [2]:
BOARD_SIZE = config()["game"]["board_size"]

In [127]:
model_torch = NeuralNet(config()["networks"]["default"])
model_parameters = filter(lambda p: p.requires_grad, model_torch.parameters())
print("Model size:", sum(p.numel() for p in model_parameters))

Model size: 40600909


In [129]:
from mlx.utils import tree_flatten

model_mlx = NeuralNetMLX(config()["networks"]["default"])
model_mlx.eval()

total = 0
for name, p in tree_flatten(model_mlx.parameters()):
    total += np.prod(p.shape)
print("Total parameters:", total)

compiled_model = mx.compile(model_mlx)

Total parameters: 40603245


In [62]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        if isinstance(model, NeuralNet):
            boards = torch.from_numpy(random_arrays[i]).to(device="mps", dtype=torch.float32)
            model(boards)
        elif isinstance(model, NeuralNetMLX):
            boards = mx.array(random_arrays[i], dtype=mx.float32)
            mx.eval(model(boards))
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [69]:
time_per_eval(
    num_evaluations=10000,
    batch_size=128,
    model=model_torch.to("mps"),
)

Starting...


6.182425631078122e-05

In [126]:
time_per_eval(
    num_evaluations=100000,
    batch_size=1,
    model=compiled_model,
)

Starting...


7.906708866357803e-08

In [122]:
random_array = np.random.random((128, 4, BOARD_SIZE, BOARD_SIZE))

In [110]:
compiled_model(mx.array(random_array))[0][56][2]

array(-0.0199862, dtype=float32)

In [114]:
model_mlx(mx.array(random_array))[0][56][2]

array(-0.0199861, dtype=float32)

In [115]:
model_mlx.parameters()["convolutional_block"]["layers"][0]["weight"].dtype

mlx.core.float32

In [26]:
model_mlx(torch.randn(1, 4, BOARD_SIZE, BOARD_SIZE))

TypeError: 'NeuralNetMLX' object is not callable